<a href="https://colab.research.google.com/github/elzosantos/modelRecomendationHotels/blob/main/Sistema_de_Recomenda%C3%A7%C3%A3o_Booking_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sistema de Recomendação por similaridade


## 1º Problema de Negócio:

Seu problema é desenvolver um sistema de recomendação para hoteis para inserir no aplicativo

##2º Análise Exploratória de Dados

In [55]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re
import random


In [119]:
import plotly
import plotly.offline as py
import plotly.graph_objs as go # criará de fato os gráficos
from plotly.offline import plot, iplot
import cufflinks as cf # para conectar o plotly ao pandas
cf.go_offline()
plotly.offline.init_notebook_mode(connected = True)
pd.options.display.max_columns = 30
cf.set_config_file(world_readable=True, theme='solar')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import plotly.io as pio
pio.renderers




Renderers configuration
-----------------------
    Default renderer: 'plotly_mimetype+notebook_connected'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']

In [120]:
pio.renderers.default = 'colab'

In [123]:
df = pd.read_csv("/content/sample_data/Seattle_Hotels.csv", sep=";", encoding="latin-1")

In [124]:
df.head()

,name,address,desc
0,Hilton Garden Seattle Downtown,"1821 Boren Avenue, Seattle Washington 98101 USA","Located on the southern tip of Lake Union, the..."
1,Sheraton Grand Seattle,"1400 6th Avenue, Seattle, Washington 98101 USA","Located in the city's vibrant core, the Sherat..."
2,Crowne Plaza Seattle Downtown,1113 6th Ave Seattle WA 98101,"Located in the heart of downtown Seattle, the ..."
3,Kimpton Hotel Monaco Seattle,"1101 4th Ave, Seattle, WA98101",What?s near our hotel downtown Seattle locatio...
4,The Westin Seattle,"1900 5th Avenue, Seattle, Washington 98101 USA",Situated amid incredible shopping and iconic a...


In [125]:
df.shape

(152, 3)

In [126]:
df = df.dropna(subset=["desc"])

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149 entries, 0 to 151
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     149 non-null    object
 1   address  149 non-null    object
 2   desc     149 non-null    object
dtypes: object(3)
memory usage: 4.7+ KB


In [110]:
df.shape

(149, 3)

In [128]:
df.isnull()

,name,address,desc
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
147,False,False,False
148,False,False,False
149,False,False,False
150,False,False,False


In [129]:
print('No dataset', len(df), 'foram encontrados')

No dataset 149 foram encontrados


In [130]:
def print_description(index):
    example = df[df.index == index][['desc', 'name']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Name:', example[1])

In [131]:
print_description(5)

More than just a hotel, The Paramount Hotel Seattle summons the feel of cozy, elegant and luxurious private residence with friendly hosts, the most comfortable beds in Seattle and an award-winning Asian restaurant right downstairs. Youll fall in love with the simple luxury and charm of our boutique hotel, from the warm, inviting wood finishes, comfy seating areas and fireplace in the lobby to the classically decorated rooms and suites appointed with a dash of urban flair. Our central downtown location puts you in good company: one block from the Washington State Convention Center and walking distance to Pike Place Market, the Seattle Art Museum and diverse shopping, cafes and restaurants. There are many reasons The Paramount Hotel is annually ranked among the top five hotels in downtown Seattle, Washington. Why not come find out yours? 
Name: The Paramount Hotel Seattle


In [132]:
print_description(50)

Welcome to the Crowne Plaza Seattle Airport Located across the sky bridge from Seattle Tacoma International Airport and adjacent to the train station, where the light rail train leaves every 12 minutes to downtown Seattle. Take the train to visit the Space Needle, Pike Place Market, Westlake Shopping Center, Nordstroms flag ship store, Century Link, Safeco Field, and the University of Washington.Just beyond our front doors are headquarters of Fortune 500 companies such as Boeing, Microsoft, Amazon, Apple, Google, Tableaux, Facebook and Starbucks. Seattle offers everything from professional sports to hiking up Mt. Rainier, leisure guests planning cruises from Seattle love our hotel's "Park, Stay and Go" package. Join us in Reflections Bar and Grill for a Pacific Northwest dining experience featuring local beers and wine with light bites.Crowne Plaza Seattle Airport has over 12,000 square feet of private event space onsite, with 7 meeting rooms, and two Ballrooms, including our 5,000 sq

**Visualize Token (vocabulary) Frequency Distribution Before Removing Stop Words**

In [133]:
%matplotlib inline
def get_top_n_words(corpus, n=None):
   vec = CountVectorizer().fit(corpus)
   bag_of_words = vec.transform(corpus)
   sum_words = bag_of_words.sum(axis=0)
   words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
   words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
   return words_freq[:n]

common_words = get_top_n_words(corpus=df['desc'], n=10)
df1 = pd.DataFrame(common_words, columns = ['desc','count'])
df1.groupby('desc').sum()['count'].sort_values().iplot(kind='barh', yTitle='Count', linecolor='black', title='Top 20 palavras')


In [134]:
%matplotlib inline
def get_top_n_words(corpus, n=None):
   vec = CountVectorizer(stop_words='english').fit(corpus)
   bag_of_words = vec.transform(corpus)
   sum_words = bag_of_words.sum(axis=0)
   words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
   words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
   return words_freq[:n]

common_words = get_top_n_words(corpus=df['desc'], n=20)
df2 = pd.DataFrame(common_words, columns = ['desc','count'])
df2.groupby('desc').sum()['count'].sort_values().iplot(kind='barh', yTitle='Count', linecolor='black', title='Top 20 palavras')


**Bigramas** - 2 termos, Distribuição de frequencia antes de remover StopWord





In [135]:
%matplotlib inline
def get_top_n_words(corpus, n=None):
   vec = CountVectorizer(ngram_range=(2,2)).fit(corpus)
   bag_of_words = vec.transform(corpus)
   sum_words = bag_of_words.sum(axis=0)
   words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
   words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
   return words_freq[:n]

common_words = get_top_n_words(corpus=df['desc'], n=20)
df2 = pd.DataFrame(common_words, columns = ['desc','count'])
df2.groupby('desc').sum()['count'].sort_values().iplot(kind='barh', yTitle='Count', linecolor='black', title='Top 20 palavras')


**Brigrama** - 2 Teremos, Distribuição de Frequencia após remover StopWords





In [136]:
%matplotlib inline
def get_top_n_words(corpus, n=None):
   vec = CountVectorizer(ngram_range=(2,2), stop_words='english').fit(corpus)
   bag_of_words = vec.transform(corpus)
   sum_words = bag_of_words.sum(axis=0)
   words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
   words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
   return words_freq[:n]

common_words = get_top_n_words(corpus=df['desc'], n=20)
df2 = pd.DataFrame(common_words, columns = ['desc','count'])
df2.groupby('desc').sum()['count'].sort_values().iplot(kind='barh', yTitle='Count', linecolor='black', title='Top 20 palavras')

**Trigramas** - 3 termos, Distribuição de frequencia antes de remover StopWord

In [137]:
%matplotlib inline
def get_top_n_words(corpus, n=None):
   vec = CountVectorizer(ngram_range=(3,3)).fit(corpus)
   bag_of_words = vec.transform(corpus)
   sum_words = bag_of_words.sum(axis=0)
   words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
   words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
   return words_freq[:n]

common_words = get_top_n_words(corpus=df['desc'], n=20)
df2 = pd.DataFrame(common_words, columns = ['desc','count'])
df2.groupby('desc').sum()['count'].sort_values().iplot(kind='barh', yTitle='Count', linecolor='black', title='Top 20 palavras')


**Trigrama** - 3 Termos, Distribuição de Frequencia após remover StopWords

In [138]:
%matplotlib inline
def get_top_n_words(corpus, n=None):
   vec = CountVectorizer(ngram_range=(3,3), stop_words='english').fit(corpus)
   bag_of_words = vec.transform(corpus)
   sum_words = bag_of_words.sum(axis=0)
   words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
   words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
   return words_freq[:n]

common_words = get_top_n_words(corpus=df['desc'], n=20)
df2 = pd.DataFrame(common_words, columns = ['desc','count'])
df2.groupby('desc').sum()['count'].sort_values().iplot(kind='barh', yTitle='Count', linecolor='black', title='Top 20 palavras')


Hotel descripition - Comprimento da Descrição do Hotel

In [139]:
df['word_count'] = df['desc'].apply(lambda x: len(str(x).split(" ")))


In [140]:
df

,name,address,desc,word_count
0,Hilton Garden Seattle Downtown,"1821 Boren Avenue, Seattle Washington 98101 USA","Located on the southern tip of Lake Union, the...",23
1,Sheraton Grand Seattle,"1400 6th Avenue, Seattle, Washington 98101 USA","Located in the city's vibrant core, the Sherat...",152
2,Crowne Plaza Seattle Downtown,1113 6th Ave Seattle WA 98101,"Located in the heart of downtown Seattle, the ...",9
3,Kimpton Hotel Monaco Seattle,"1101 4th Ave, Seattle, WA98101",What?s near our hotel downtown Seattle locatio...,10
4,The Westin Seattle,"1900 5th Avenue, Seattle, Washington 98101 USA",Situated amid incredible shopping and iconic a...,151
...,...,...,...,...
147,The Halcyon Suite Du Jour,"1125 9th Ave W, Seattle, WA 98119","Located in Queen Anne district, The Halcyon Su...",55
148,Vermont Inn,"2721 4th Ave, Seattle, WA 98121",Just a block from the world famous Space Needl...,98
149,Stay Alfred on Wall Street,"2515 4th Ave, Seattle, WA 98121",Stay Alfred on Wall Street resides in the hear...,143
150,Pike's Place Lux Suites by Barsala,"2nd Ave and Stewart St, Seattle, WA 98101",The perfect marriage of heightened convenience...,57


In [142]:
desc_lengths = list(df['word_count'])
print("Number of descriptions:", len(desc_lengths))
print("Average word count:", np.average(desc_lengths), "words")
print("Minimum word count:", np.min(desc_lengths), "words")
print("Maximum word count:", np.max(desc_lengths), "words")


Number of descriptions: 149
Average word count: 155.18120805369128 words
Minimum word count: 9 words
Maximum word count: 492 words


In [143]:
%matplotlib inline
df['word_count'].iplot(kind='hist', xTitle='Word Count',bins=50, linecolor='black', title='Contagem de palavras na descrição do Hotel')

##3º Pre-processing dos Dados - Hotel Descrição Texto


In [145]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [146]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

df['desc_clean'] = df['desc'].apply(clean_text)

In [147]:
def print_description(index):
    example = df[df.index == index][['desc_clean', 'name']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Name:', example[1])
print_description(7)

downtown seattle destination hotel steps everywhere want seattle style motif local welcome libation rooftop lounge views across city touchstones areas sweeping landscape rich arts music culture infuse surroundings newly renovated rooms feel like modern residences natural hardwoods colors inspired pacific northwest youll find regions culinary bounty reflected menus frolik kitchen + cocktails adjoining fifth floor patio join us 5th avenue pike union shopping pike place market waterfront convention center seattle monorail light rail airport stroll away
Name: Motif Seattle


In [148]:
print_description(100)

budget seattle looking something different historic charm home away home atmosphere baroness sure make feel like one family conveniently located first hill proud part virginia mason hospital campus minutes harborview medical center swedish hospital baroness hotel great option short long term medical patient family stays whether visiting areas worldclass medical facilities budget vacation goal ensure wonderful stay guest amenities complimentary internet access two twin one two queen studios mini fridge microwave two twin one queen suites full kitchens laundry facilities available flat screen cable television hbo complimentary local calls ice vending machines located lobby coffee maker hairdryers guestrooms room service available seven days week rhododendron cafe limited wheelchair accessibility guest library business center printing fax services available 100 nonsmoking pet free rooms air conditioned fans available selfparking available virginia mason hospital fee
Name: The Baroness Hot

In [149]:
df.set_index('name', inplace=True)

In [150]:
df

,address,desc,word_count,desc_clean
name,,,,
Hilton Garden Seattle Downtown,"1821 Boren Avenue, Seattle Washington 98101 USA","Located on the southern tip of Lake Union, the...",23,located southern tip lake union hilton garden ...
Sheraton Grand Seattle,"1400 6th Avenue, Seattle, Washington 98101 USA","Located in the city's vibrant core, the Sherat...",152,located citys vibrant core sheraton grand seat...
Crowne Plaza Seattle Downtown,1113 6th Ave Seattle WA 98101,"Located in the heart of downtown Seattle, the ...",9,located heart downtown seattle awardwinning
Kimpton Hotel Monaco Seattle,"1101 4th Ave, Seattle, WA98101",What?s near our hotel downtown Seattle locatio...,10,whats near hotel downtown seattle location better
The Westin Seattle,"1900 5th Avenue, Seattle, Washington 98101 USA",Situated amid incredible shopping and iconic a...,151,situated amid incredible shopping iconic attra...
...,...,...,...,...
The Halcyon Suite Du Jour,"1125 9th Ave W, Seattle, WA 98119","Located in Queen Anne district, The Halcyon Su...",55,located queen anne district halcyon suite du j...
Vermont Inn,"2721 4th Ave, Seattle, WA 98121",Just a block from the world famous Space Needl...,98,block world famous space needle seattle center...
Stay Alfred on Wall Street,"2515 4th Ave, Seattle, WA 98121",Stay Alfred on Wall Street resides in the hear...,143,stay alfred wall street resides heart belltown...


##4º Maquina Preditiva com ML

In [156]:
#ngram 1,3 faz unigrama, bigrama e trigrama
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0.0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['desc_clean'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [157]:
indices = pd.Series(df.index)

In [159]:
indices[:3]

,name
0,Hilton Garden Seattle Downtown
1,Sheraton Grand Seattle
2,Crowne Plaza Seattle Downtown


In [160]:
def recomendations(name, cosine_similarities = cosine_similarities):
   recomendated_hotels = []

   idx = indices[indices == name].index[0]

   score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

   top_10_indexes = list(score_series.iloc[1:11].index)

   for i in top_10_indexes:
       recomendated_hotels.append(list(df.index)[i])
   return recomendated_hotels

In [161]:
recomendations('Hilton Seattle Airport & Conference Center')

['Embassy Suites by Hilton Seattle Tacoma International Airport',
 'DoubleTree by Hilton Hotel Seattle Airport',
 'Seattle Airport Marriott',
 'Econo Lodge SeaTac Airport North',
 'Four Points by Sheraton Downtown Seattle Center',
 'Knights Inn Tukwila',
 'Econo Lodge Renton-Bellevue',
 'Hampton Inn Seattle/Southcenter',
 'Radisson Hotel Seattle Airport',
 'Home2 Suites by Hilton Seattle Airport']